In [1]:
from sklearn.model_selection import train_test_split
import nltk
import numpy as np
import string

이상한 나라의 엘리스 소설을 읽어온다.&전처리!

In [2]:
with open('alice_in_wonderland.txt', 'r') as content_file:
    content = content_file.read()
#                                         특수 문자를 공백 문자로 치환
content2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in content]).split())
 
tokens = nltk.word_tokenize(content2)
tokens = [word.lower() for word in tokens if len(word)>=2]

Trigram list

In [3]:
N = 3 # 3개 단어씩 토큰묶기(소설 전체 대상)
quads = list(nltk.ngrams(tokens, N))

In [4]:
quads[:3] # sentence 고려는 안하고 그냥 간단하게! (그냥 다음 문장으로 넘어가게 됨)

[('alice', 'adventures', 'in'),
 ('adventures', 'in', 'wonderland'),
 ('in', 'wonderland', 'alice')]

In [5]:
newl_app = []
for ln in quads:
    newl = " ".join(ln)
    newl_app.append(newl) # 리스트화

In [6]:
newl_app[:3]

['alice adventures in', 'adventures in wonderland', 'in wonderland alice']

Vectorizing the words

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() #TFIDF아닌 카운트벡터라이저!

x_trigm = []
y_trigm = []

for l in newl_app:
    x_str = " ".join(l.split()[0:N-1])  # trigram 중 앞 부분 2 단어. 
    y_str = l.split()[N-1]              # trigram 중 마지막 1 단어.  
    x_trigm.append(x_str)
    y_trigm.append(y_str)

x_trigm_check = vectorizer.fit_transform(x_trigm).todense()
y_trigm_check = vectorizer.fit_transform(y_trigm).todense()

In [8]:
x_trigm_check[0] # 수치화 완성

matrix([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Dictionaries from word to integer and integer to word

In [10]:
dictnry = vectorizer.vocabulary_ # _ 해주면 전체 voca로 word2idx해줌
rev_dictnry = {v:k for k,v in dictnry.items()}
#리버스 idx2word
X = np.array(x_trigm_check) # 수치화 완료
Y = np.array(y_trigm_check)
                                                    #학습, 시험 데이터 분리
Xtrain, Xtest, Ytrain, Ytest,xtrain_tg,xtest_tg = train_test_split(X, Y,x_trigm, test_size=0.3,random_state=42)
#진짜 notwork에 사용되는 Xtrain!
print("X Train shape",Xtrain.shape, "Y Train shape" , Ytrain.shape) 
# 두 단어가 섞인 []가 17947/voca 수 =2559개! 어떠한 단어가 쓰였는가 표시
print("X Test shape",Xtest.shape, "Y Test shape" , Ytest.shape)

X Train shape (17947, 2559) Y Train shape (17947, 2559)
X Test shape (7692, 2559) Y Test shape (7692, 2559)


Model Building

In [11]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

np.random.seed(42)

BATCH_SIZE = 128
NUM_EPOCHS = 10
            # input(batch_shape=(NONE, )) 배치쓰면 NONE으로 줄 수 있음! <-2D구조
input_layer = Input(shape = (Xtrain.shape[1],))
first_layer = Dense(1000, activation='relu')(input_layer) #히든 수 1000개!
first_dropout = Dropout(rate = 0.5)(first_layer) # W의 50%만 버려!

second_layer = Dense(800, activation='relu')(first_dropout)# 이 위로 800개 쌓음

third_layer = Dense(1000, activation='relu')(second_layer) # 또 1000개 쌓음
third_dropout = Dropout(rate = 0.5)(third_layer)

fourth_layer = Dense(Ytrain.shape[1], activation='softmax')(third_dropout)


history = Model(input_layer, fourth_layer)
history.compile(optimizer = "adam",loss="categorical_crossentropy", metrics=["accuracy"])

print (history.summary())

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2559)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              2560000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 800)               800800    
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              801000    
_________________________________________________________________
dropout_2 (Dropout)  

6,723,359개의 W를 update해야 한다  
..    
Model Training

In [12]:
history.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=1,validation_split = 0.2)

Instructions for updating:
Use tf.cast instead.
Train on 14357 samples, validate on 3590 samples
Epoch 1/10
14357/14357 [==============================] - 16s 1ms/step - loss: 6.3384 - acc: 0.0545 - val_loss: 6.1011 - val_acc: 0.0680
Epoch 2/10
14357/14357 [==============================] - 13s 897us/step - loss: 5.8985 - acc: 0.0685 - val_loss: 6.0288 - val_acc: 0.0730
Epoch 3/10
14357/14357 [==============================] - 13s 922us/step - loss: 5.6758 - acc: 0.0846 - val_loss: 5.9798 - val_acc: 0.0858
Epoch 4/10
14357/14357 [==============================] - 13s 896us/step - loss: 5.4240 - acc: 0.1066 - val_loss: 5.9822 - val_acc: 0.0933
Epoch 5/10
14357/14357 [==============================] - 13s 884us/step - loss: 5.1693 - acc: 0.1327 - val_loss: 5.9730 - val_acc: 0.1000
Epoch 6/10
14357/14357 [==============================] - 13s 897us/step - loss: 4.9345 - acc: 0.1507 - val_loss: 5.9924 - val_acc: 0.1047
Epoch 7/10
14357/14357 [==============================] - 13s 888us/ste

Model Prediction

In [ ]:
Y_pred = history.predict(Xtest)

Sample check on Test data

In [64]:
print ("Prior bigram words","| Actual","| Predicted","\n")
#확률 순으로 추천단어 5개 추출로 뽑아주게 변경
for i in range(10):
    print (i,xtest_tg[i], "|", rev_dictnry[np.argmax(Ytest[i])], "| ", end='')
    idx = np.flipud(Y_pred[i].argsort())[:5]
    for k in idx:
        print(rev_dictnry[k], ' ', end='')
    print()

Prior bigram words | Actual | Predicted 

0 the evening | beautiful | beautiful  turtle  rabbit  hall  little  
1 slipped in | like | and  in  the  with  half  
2 alice swallowing | down | good  her  very  little  not  
3 an encouraging | tone | of  to  but  again  voice  
4 waistcoat pocket | or | went  music  and  sat  repeated  
5 she went | on | on  down  back  hunting  up  
6 that she | knew | was  had  would  got  said  
7 down on | her | one  the  his  their  her  
8 dormouse went | on | on  down  after  without  back  
9 soup soup | of | you  said  and  to  in  
